In [1]:
! pip install git+https://github.com/python-control/python-control@601b58152080d89575cc677474ec7714e1a34ee2
import control
import numpy as np
from scipy.interpolate import interp1d

  Cloning https://github.com/python-control/python-control (to revision 601b58152080d89575cc677474ec7714e1a34ee2) to /tmp/pip-req-build-ja4wxewz
  Stored in directory: /tmp/pip-ephem-wheel-cache-egv8oqx8/wheels/d5/fa/66/d16620c4f2aa2183831970718f2fe9cf08dbca08c714a76668
Successfully built control


In [2]:
NUM_PROFILES = 100
MAX_TIME = 60
T_sp = 0
K_c, tau_I, tau_D, tau_c = [5688.814947611792, 0.0002280222206125743, 0.4941522520819933, 0.006245557968152839]


# Disturbance profiles were uploaded to Github so markers can access them through this notebook. Let's get them.
!rm -rf CHBE356-data
!git clone https://github.com/TimHillmer/CHBE356-data/

# obtain a numerically ordered list of the paths to the disturbance profiles
import glob
filenames = glob.glob('CHBE356-data/Disturbance_profiles/*.csv')
filenames.sort()

# obtain an array containing all disturbance profiles (data in [profile#, timevalue] format)
import pandas as pd
df = pd.concat([pd.read_csv(files).transpose() for files in filenames], ignore_index = True)
# each disturbance profile is a *row*
Ti_data = df.values

Cloning into 'CHBE356-data'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 170 (delta 13), reused 0 (delta 0), pack-reused 106
Receiving objects: 100% (170/170), 992.74 KiB | 3.75 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [0]:
def simulate(i, K_c, tau_I, tau_D, tau_c):
    time_data = np.linspace(0, MAX_TIME-1, MAX_TIME)                                                                                                                                                                                                           
    T_i_data = Ti_data[i]
    f = interp1d(time_data, T_i_data)
    points_per_seond = 100
    time = np.linspace(0, MAX_TIME-1, MAX_TIME*points_per_seond)
    T_i = f(time)

    s = control.tf([1,0],[0,1])
    G_p = 1/(s**2 + s + 1)
    G_d = (s+1)/(s**2 + s + 1)
    G_c = K_c * (1+ 1/(tau_I*s) + (tau_D*s)/(tau_c*s + 1))
    sys_D = G_d / (1 + G_p * G_c)

    Tsp = 0
    _, T, _ = control.forced_response(sys_D, time, T_i)
    _, Q, _ = control.forced_response(G_c, time, Tsp - T)
    error = (sum(abs(T)) + 0.2*sum(abs(Q)))/points_per_seond
    if sum(abs(T)>= 5) or sum(abs(Q)>= 5):
        error = 1e6
    return error

In [4]:
total_error = 0
for i in range(NUM_PROFILES):
    error = simulate(i, K_c, tau_I, tau_D, tau_c)
    total_error += error
print(f'Average error: {total_error / NUM_PROFILES}')

Average error: 6.603490145150943
